In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811636 sha256=64c9c637a2220a17a1f9dbd221b38de51b40006446ef4f646c654c0e1423ecb5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd

from surprise import Dataset, SVD
from collections import defaultdict
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [12]:
with open("/root/.surprise_data/ml-100k/ml-100k/u.info", "r") as f:
    print(f.read())

943 users
1682 items
100000 ratings



In [13]:
trainset, testset = train_test_split(data, test_size=0.2)

In [18]:
for x in trainset.all_ratings():
    print(x)
    break

(0, 0, 4.0)


In [19]:
testset[:5]

[('311', '1116', 3.0),
 ('877', '306', 3.0),
 ('43', '64', 5.0),
 ('916', '246', 5.0),
 ('290', '144', 3.0)]

In [21]:
algo = SVD()

In [22]:
algo.fit(trainset)

In [42]:
preds = algo.test(testset)

In [43]:
testset[:5]

[('311', '1116', 3.0),
 ('877', '306', 3.0),
 ('43', '64', 5.0),
 ('916', '246', 5.0),
 ('290', '144', 3.0)]

In [44]:
preds[:5]

[Prediction(uid='311', iid='1116', r_ui=3.0, est=3.3669287362039766, details={'was_impossible': False}),
 Prediction(uid='877', iid='306', r_ui=3.0, est=3.8716415415383416, details={'was_impossible': False}),
 Prediction(uid='43', iid='64', r_ui=5.0, est=4.563515382937223, details={'was_impossible': False}),
 Prediction(uid='916', iid='246', r_ui=5.0, est=3.996719183413975, details={'was_impossible': False}),
 Prediction(uid='290', iid='144', r_ui=3.0, est=3.659036784612865, details={'was_impossible': False})]

In [45]:
from surprise import accuracy

In [49]:
accuracy.mae(preds)

MAE:  0.7384


0.7384403354336518

In [50]:
accuracy.mae(preds[:1])

MAE:  0.3669


0.3669287362039766

In [51]:
abs(3.0 - 3.3669287362039766)

0.3669287362039766

In [53]:
# припущення що trainset це всі дані що в нас є
predictions = algo.test(trainset.build_anti_testset())

In [54]:
1000 * 1600 - 100000

1500000

In [55]:
len(predictions)

1487266

In [56]:
predictions[:5]

[Prediction(uid='69', iid='191', r_ui=3.5309875, est=4.581758489715941, details={'was_impossible': False}),
 Prediction(uid='69', iid='544', r_ui=3.5309875, est=3.488702138966563, details={'was_impossible': False}),
 Prediction(uid='69', iid='852', r_ui=3.5309875, est=3.4313448829236233, details={'was_impossible': False}),
 Prediction(uid='69', iid='483', r_ui=3.5309875, est=4.503280507037653, details={'was_impossible': False}),
 Prediction(uid='69', iid='185', r_ui=3.5309875, est=4.289521640459391, details={'was_impossible': False})]

In [58]:
# d = {"69": [("191", 4.5), ("544", 3.48), ...], ...}

top_n = defaultdict(list)
for uid, iid, r_ui, est, details in predictions:
    top_n[uid].append((iid, est))

In [65]:
for uid, user_ratings in top_n.items():
    top_n[uid] = sorted(user_ratings, key=lambda x: -x[1])

In [70]:
for uid, user_ratings in list(top_n.items())[:10]:
    print(f"{uid=}", [x[0] for x in user_ratings[:5]])

uid='69' ['127', '318', '480', '187', '511']
uid='721' ['174', '96', '316', '483', '168']
uid='666' ['285', '493', '474', '9', '408']
uid='13' ['169', '519', '50', '963', '408']
uid='272' ['64', '408', '169', '114', '615']
uid='248' ['483', '408', '269', '169', '318']
uid='889' ['496', '169', '30', '176', '313']
uid='399' ['185', '657', '483', '515', '513']
uid='198' ['50', '408', '187', '483', '12']
uid='497' ['174', '199', '483', '134', '170']


In [73]:
top_100 = defaultdict(list)

for uid, user_ratings in top_n.items():
    top_100[uid] = user_ratings[:100]

In [75]:
with open("rec_data.txt", "w") as file:
    file.write(str(top_100))

In [81]:
import json


with open("rec_data.json", "w") as file:
    json.dump(top_100, file)